In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import random
import pickle
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import pandas as pd
import numpy as np
import wave
from io import BytesIO
from IPython.display import Audio, display
from transformers import RobertaTokenizer, RobertaModel, Wav2Vec2FeatureExtractor
from transformers.models.wavlm import WavLMModel

In [3]:
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torch.utils.tensorboard import SummaryWriter


In [4]:
database_dir = '/content/drive/MyDrive/DAIC_WOZ'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [5]:
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("microsoft/wavlm-large")
wavlm_model = WavLMModel.from_pretrained("microsoft/wavlm-large").to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/214 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.22k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

In [6]:
from transformers import AutoTokenizer, AutoModel
model_name = "roberta-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
text_model = AutoModel.from_pretrained(model_name)
text_model = text_model.to(device)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
def get_sentence_level_text_embedding(sentence, tokenizer, text_model, device):
    # Generate attention_mask for text embeddings
    inputs = tokenizer(
        sentence,
        return_tensors="pt",
        padding=True,
        truncation=True,
        return_attention_mask=True
    ).to(device)

    with torch.no_grad():
        outputs = text_model(**inputs)
    # Take [CLS] token embedding
    sentence_embedding = outputs.last_hidden_state[:, 0, :]
    return sentence_embedding.squeeze(0)

def get_audio_embedding(audio_segment, feature_extractor, wavlm_model, device):
    audio_data = np.array(audio_segment.get_array_of_samples()).astype(np.int16)
    audio_data_tensor = torch.tensor(audio_data, dtype=torch.float32).to(device)

    # Generate attention_mask for audio embeddings
    inputs = feature_extractor(
        audio_data_tensor,
        sampling_rate=audio_segment.frame_rate,
        return_tensors="pt",
        padding=True,
        return_attention_mask=True
    )
    inputs = {key: val.to(device) for key, val in inputs.items()}

    with torch.no_grad():
        outputs = wavlm_model(**inputs)

    # outputs.last_hidden_state: (1, time_steps, embedding_dim)
    audio_embeddings = outputs.last_hidden_state.squeeze(0)
    # Average pooling over time
    audio_embedding_vector = audio_embeddings.mean(dim=0)
    return audio_embedding_vector
def load_label_dict(label_csv_path):
    """
    Loads a CSV file that contains at least these columns: "Participant_ID", "PHQ8_Binary".
    Converts Participant_ID to an integer, then to a string to match how we handle directory-based IDs.
    """
    df = pd.read_csv(label_csv_path)
    label_dict = {}
    for _, row in df.iterrows():
        base_id_str = str(int(row["Participant_ID"]))  # Ensures "315.0" becomes "315"
        label = int(row["PHQ8_Binary"])
        label_dict[base_id_str] = label
        print(f"Loaded label for {base_id_str}: {label}")
    return label_dict


In [10]:
class DAICDataset(Dataset):
    def __init__(self, root_dir, label_csv_path, tokenizer, text_model, feature_extractor, wavlm_model, device, embeddings_dir=None):
        """
        root_dir: path to the split directory (train, validate, or test) containing session folders.
        label_csv_path: path to the CSV file containing Participant_ID and PHQ8_Binary columns.
        embeddings_dir: optional directory to save/load precomputed embeddings.
        """
        self.root_dir = root_dir
        self.tokenizer = tokenizer
        self.text_model = text_model
        self.feature_extractor = feature_extractor
        self.wavlm_model = wavlm_model
        self.device = device
        self.embeddings_dir = embeddings_dir

        # Load the label dictionary
        self.label_dict = load_label_dict(label_csv_path)

        self.session_ids = [d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))]
        self.data = []
        self._prepare_data()

    def _prepare_data(self):
        # If embeddings_dir is provided, create split-specific dir
        split_name = os.path.basename(self.root_dir)  # 'train', 'validate', 'test'
        if self.embeddings_dir is not None:
            split_embeddings_dir = os.path.join(self.embeddings_dir, split_name)
            os.makedirs(split_embeddings_dir, exist_ok=True)
        else:
            split_embeddings_dir = None

        for session_id in self.session_ids:
            base_id_str = session_id.split('_')[0]  # e.g. "315" from "315_P"
            print(base_id_str)
            # Check if we have precomputed embeddings
            embed_file_path = None
            if split_embeddings_dir is not None:
                embed_file_path = os.path.join(split_embeddings_dir, f"{session_id}.pt")
                if os.path.exists(embed_file_path):
                    # Load precomputed embeddings
                    saved_data = torch.load(embed_file_path)
                    text_sequence = saved_data['text_sequence']
                    audio_sequence = saved_data['audio_sequence']
                    label = saved_data['label']
                    self.data.append((text_sequence, audio_sequence, label))
                    continue

            # If we don't have precomputed embeddings or embeddings_dir not used, process data
            transcript_file, audio_file = self._find_files(session_id)
            if transcript_file is None or audio_file is None:
                print(f"Transcript or audio missing for {session_id}, skipping.")
                continue

            filtered_df = self._load_and_filter_transcript(transcript_file)
            if filtered_df is None or len(filtered_df) == 0:
                # No valid transcripts after filtering Ellie
                continue

            if not os.path.exists(audio_file):
                print(f"Audio file not found: {audio_file}")
                continue

            full_audio = AudioSegment.from_wav(audio_file)

            aligned_data = []
            for _, row in filtered_df.iterrows():
                try:
                    start_time = float(row["start_time"])
                    stop_time = float(row["stop_time"])
                    sentence = row["value"]
                    start_ms = int(start_time * 1000)
                    stop_ms = int(stop_time * 1000)
                    audio_segment = full_audio[start_ms:stop_ms]

                    aligned_data.append({
                        "start_time": start_time,
                        "stop_time": stop_time,
                        "sentence": sentence,
                        "audio_segment": audio_segment
                    })
                except ValueError:
                    continue

            if len(aligned_data) == 0:
                # No valid sentences
                continue
            min_duration = 0.1  # minimum duration in seconds

            if (len(audio_segment) / 1000.0) < min_duration:
            # Skip this segment
                continue


            text_embeddings = []
            audio_embeddings = []
            for entry in aligned_data:
                text_emb = get_sentence_level_text_embedding(entry["sentence"], self.tokenizer, self.text_model)
                audio_emb = get_audio_embedding(entry["audio_segment"], self.feature_extractor, self.wavlm_model)
                text_embeddings.append(text_emb)
                audio_embeddings.append(audio_emb)

            if len(text_embeddings) == 0:
                continue

            # Stack embeddings: [num_sentences, embedding_dim]
            text_sequence = torch.stack(text_embeddings, dim=0)
            audio_sequence = torch.stack(audio_embeddings, dim=0)

            # Move them to CPU before storing in self.data
            text_sequence = text_sequence.cpu()
            audio_sequence = audio_sequence.cpu()

            if base_id_str not in self.label_dict:
                print(f"No label found for {base_id_str}")
                label = 0
            else:
                label = self.label_dict[base_id_str]
                print(f"{base_id_str}: {label}")

            # Save embeddings if path is given
            if embed_file_path is not None:
                torch.save({
                    'text_sequence': text_sequence,
                    'audio_sequence': audio_sequence,
                    'label': label
                }, embed_file_path)

            self.data.append((text_sequence, audio_sequence, label))


    def _find_files(self, session_id):
        session_path = os.path.join(self.root_dir, session_id)
        transcript_file = None
        audio_file = None
        for fn in os.listdir(session_path):
            if fn.endswith("_TRANSCRIPT.csv"):
                transcript_file = os.path.join(session_path, fn)
            if fn.endswith("_AUDIO.wav"):
                audio_file = os.path.join(session_path, fn)
        return transcript_file, audio_file

    def _load_and_filter_transcript(self, transcript_file):
        df = pd.read_csv(transcript_file, usecols=[0], header=None)
        df_split = df[0].str.split('\t', expand=True)
        if df_split.shape[1] < 4:
            print(f"Transcript file {transcript_file} has unexpected format.")
            return None
        df_split.columns = ["start_time", "stop_time", "speaker", "value"]
        filtered_df = df_split[df_split["speaker"] != "Ellie"]
        return filtered_df

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

In [11]:
def collate_fn(batch):
    text_seqs = [item[0] for item in batch]  # CPU tensors
    audio_seqs = [item[1] for item in batch] # CPU tensors
    labels = [item[2] for item in batch]

    max_len_text = max(seq.size(0) for seq in text_seqs)
    max_len_audio = max(seq.size(0) for seq in audio_seqs)

    padded_text = []
    for seq in text_seqs:
        if seq.size(0) < max_len_text:
            diff = max_len_text - seq.size(0)
            pad_tensor = torch.zeros(diff, seq.size(1))  # should be CPU
            # Debug prints for text sequence
            #print("Text seq device before cat:", seq.device)
            #print("Text pad_tensor device before cat:", pad_tensor.device)
            seq = torch.cat([seq, pad_tensor], dim=0)  # Both seq and pad_tensor should be on CPU
            #print("Text seq device after cat:", seq.device)
        padded_text.append(seq.unsqueeze(0))
    padded_text = torch.cat(padded_text, dim=0)  # [batch_size, max_len_text, D]

    padded_audio = []
    for seq in audio_seqs:
        if seq.size(0) < max_len_audio:
            diff = max_len_audio - seq.size(0)
            pad_tensor = torch.zeros(diff, seq.size(1))  # should be CPU
            # Debug prints for audio sequence
            #print("Audio seq device before cat:", seq.device)
            #print("Audio pad_tensor device before cat:", pad_tensor.device)
            seq = torch.cat([seq, pad_tensor], dim=0)  # Both seq and pad_tensor should be on CPU
            #print("Audio seq device after cat:", seq.device)
        padded_audio.append(seq.unsqueeze(0))
    padded_audio = torch.cat(padded_audio, dim=0) # [batch_size, max_len_audio, D]

    labels = torch.tensor(labels, dtype=torch.long)  # CPU

    #print("Final shapes:", padded_text.shape, padded_audio.shape, labels.shape)
    #print("Final devices:", padded_text.device, padded_audio.device, labels.device)

    return padded_text, padded_audio, labels


In [12]:
train_label_csv = "/content/drive/MyDrive/pkl/train_set.csv"
val_label_csv = "/content/drive/MyDrive/pkl/validation_set.csv"
test_label_csv = "/content/drive/MyDrive/pkl/full_test_split.csv"

train_dir = os.path.join(database_dir, 'train')
val_dir = os.path.join(database_dir, 'validate')
test_dir = os.path.join(database_dir, 'test')


In [13]:
train_embedding_dir = '/content/drive/MyDrive/Embeddings_sub/train'
val_embedding_dir = '/content/drive/MyDrive/Embeddings_sub/validate'
test_embedding_dir = '/content/drive/MyDrive/Embeddings_sub/test'

In [14]:
# this class loads the pre-computed .pt files to be stored in internal list
class PrecomputedEmbeddingsDataset(Dataset):
    def __init__(self, embeddings_dir):
        """
        embeddings_dir: Directory containing .pt files.
        Each .pt file should contain a dictionary:
          {
            'text_sequence': torch.Tensor, # [num_sentences, text_dim]
            'audio_sequence': torch.Tensor, # [num_sentences, audio_dim]
            'label': int
          }
        """
        self.embeddings_dir = embeddings_dir
        self.data = []
        self._load_data()

    def _load_data(self):
        # Get all .pt files in the directory
        all_files = [f for f in os.listdir(self.embeddings_dir) if f.endswith('.pt')]
        if not all_files:
            print(f"No .pt files found in {self.embeddings_dir}")

        for fname in all_files:
            file_path = os.path.join(self.embeddings_dir, fname)
            try:
                saved_data = torch.load(file_path, map_location='cpu')
                text_sequence = saved_data['text_sequence']
                audio_sequence = saved_data['audio_sequence']
                label = saved_data['label']
                self.data.append((text_sequence, audio_sequence, label))
            except Exception as e:
                print(f"Error loading {file_path}: {e}")

        print(f"Loaded {len(self.data)} sessions from {self.embeddings_dir}")

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Returns a tuple: (text_sequence, audio_sequence, label)
        return self.data[idx]

def collate_fn(batch):
    text_seqs = [item[0] for item in batch]  # text embeddings
    audio_seqs = [item[1] for item in batch] # audio embeddings
    labels = [item[2] for item in batch]

    # Compute padding lengths
    max_len_text = max(seq.size(0) for seq in text_seqs) if len(text_seqs) > 0 else 0
    max_len_audio = max(seq.size(0) for seq in audio_seqs) if len(audio_seqs) > 0 else 0

    # Create masks
    text_mask = torch.zeros(len(text_seqs), max_len_text, dtype=torch.long)
    audio_mask = torch.zeros(len(audio_seqs), max_len_audio, dtype=torch.long)

    padded_text = []
    for i, seq in enumerate(text_seqs):
        original_len = seq.size(0)
        if original_len < max_len_text:
            diff = max_len_text - original_len
            pad_tensor = torch.zeros(diff, seq.size(1))
            seq = torch.cat([seq, pad_tensor], dim=0)
        padded_text.append(seq.unsqueeze(0))
        text_mask[i, :original_len] = 1

    padded_text = torch.cat(padded_text, dim=0) if len(padded_text) > 0 else torch.empty(0)

    padded_audio = []
    for i, seq in enumerate(audio_seqs):
        original_len = seq.size(0)
        if original_len < max_len_audio:
            diff = max_len_audio - original_len
            pad_tensor = torch.zeros(diff, seq.size(1))
            seq = torch.cat([seq, pad_tensor], dim=0)
        padded_audio.append(seq.unsqueeze(0))
        audio_mask[i, :original_len] = 1

    padded_audio = torch.cat(padded_audio, dim=0) if len(padded_audio) > 0 else torch.empty(0)
    labels = torch.tensor(labels, dtype=torch.long)

    return padded_text, padded_audio, text_mask, audio_mask, labels

In [15]:
train_dataset = PrecomputedEmbeddingsDataset(train_embedding_dir)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)

val_dataset = PrecomputedEmbeddingsDataset(val_embedding_dir)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)

test_dataset = PrecomputedEmbeddingsDataset(test_embedding_dir)
test_loader = DataLoader(val_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)

<ipython-input-14-a78ae2d4be15>:26: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  saved_data = torch.load(file_path, map_location='cpu')


Loaded 83 sessions from /content/drive/MyDrive/Embeddings_sub/train
Loaded 21 sessions from /content/drive/MyDrive/Embeddings_sub/validate
Loaded 47 sessions from /content/drive/MyDrive/Embeddings_sub/test


# BiLSTM

In [16]:
class BiLSTMClassifier(nn.Module):
    def __init__(
        self,
        text_dim,          # dimension of text embeddings
        audio_dim,         # dimension of audio embeddings
        hidden_dim=128,    # LSTM hidden dimension
        num_layers=2,      # number of LSTM layers
        dropout=0.3,
        use_attention=True
    ):
        super(BiLSTMClassifier, self).__init__()
        self.use_attention = use_attention

        # Bi-LSTM for text
        self.text_lstm = nn.LSTM(
            input_size=text_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            dropout=dropout,
            batch_first=True,
            bidirectional=True
        )

        # Bi-LSTM for audio
        self.audio_lstm = nn.LSTM(
            input_size=audio_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            dropout=dropout,
            batch_first=True,
            bidirectional=True
        )

        # Attention parameters (optional)
        if use_attention:
            self.text_attn = nn.Linear(hidden_dim * 2, hidden_dim * 2)
            self.audio_attn = nn.Linear(hidden_dim * 2, hidden_dim * 2)
            self.text_context_vector = nn.Parameter(torch.randn(hidden_dim * 2))
            self.audio_context_vector = nn.Parameter(torch.randn(hidden_dim * 2))

        # Final classifier:
        #   hidden_dim * 2 (bidirectional) from text + hidden_dim * 2 (bidirectional) from audio
        #   =>  hidden_dim * 4 in total (if using simple concatenation)
        self.classifier = nn.Sequential(
            nn.Linear(hidden_dim * 4, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, 2)  # binary classification => output size is 2
        )

    def forward(self, text_input, text_mask, audio_input, audio_mask):
        """
        text_input : (batch_size, seq_len_text, text_dim)
        text_mask  : (batch_size, seq_len_text)
        audio_input: (batch_size, seq_len_audio, audio_dim)
        audio_mask : (batch_size, seq_len_audio)
        """
        # Run Bi-LSTM on text
        text_output, _ = self.text_lstm(text_input)
        # text_output is (batch_size, seq_len_text, 2*hidden_dim) because it's bidirectional

        # Run Bi-LSTM on audio
        audio_output, _ = self.audio_lstm(audio_input)
        # audio_output is (batch_size, seq_len_audio, 2*hidden_dim)

        if self.use_attention:
            # Apply attention to text_output
            text_rep = self.attention_pooling(text_output, text_mask,
                                              self.text_attn, self.text_context_vector)
            # Apply attention to audio_output
            audio_rep = self.attention_pooling(audio_output, audio_mask,
                                               self.audio_attn, self.audio_context_vector)
        else:
            # Alternatively, just average over time for each modality
            text_rep = self.mean_pooling(text_output, text_mask)
            audio_rep = self.mean_pooling(audio_output, audio_mask)

        # Fuse text_rep and audio_rep by concatenation
        fused = torch.cat([text_rep, audio_rep], dim=1)  # (batch_size, 4 * hidden_dim)

        # Pass through classifier
        logits = self.classifier(fused)  # (batch_size, 2)
        return logits

    def mean_pooling(self, rnn_output, mask):
        """
        rnn_output: (batch_size, seq_len, hidden_dim * 2)
        mask:       (batch_size, seq_len)
        Returns average of valid positions only (where mask=1).
        """
        mask = mask.unsqueeze(-1)  # (batch_size, seq_len, 1)
        masked_output = rnn_output * mask  # zero out padded tokens
        sum_embeds = masked_output.sum(dim=1)  # sum across seq_len
        lengths = mask.sum(dim=1).clamp(min=1e-9)  # avoid division by zero
        mean_embeds = sum_embeds / lengths
        return mean_embeds  # (batch_size, hidden_dim*2)

    def attention_pooling(self, rnn_output, mask, attn_layer, context_vector):
        """
        An example additive attention pooling.
        rnn_output:    (batch_size, seq_len, hidden_dim * 2)
        mask:          (batch_size, seq_len)
        attn_layer:    nn.Linear -> transforms rnn_output
        context_vector: learned parameter -> shape (hidden_dim * 2)
        """
        # 1) Transform
        attn_scores = attn_layer(rnn_output)  # (batch_size, seq_len, hidden_dim*2)
        # 2) Compute scores via dot product with context_vector
        #    context_vector -> (hidden_dim*2,)
        attn_scores = torch.tanh(attn_scores)  # activation
        attn_scores = torch.matmul(attn_scores, context_vector)  # => (batch_size, seq_len)

        # 3) Mask out padded tokens
        mask = mask.float()
        attn_scores = attn_scores.masked_fill(mask == 0, float('-inf'))

        # 4) Softmax over sequence dimension
        attn_weights = F.softmax(attn_scores, dim=1)  # (batch_size, seq_len)

        # 5) Weighted sum of rnn_output
        attn_weights = attn_weights.unsqueeze(-1)  # (batch_size, seq_len, 1)
        weighted_output = rnn_output * attn_weights
        representation = weighted_output.sum(dim=1)  # (batch_size, hidden_dim*2)

        return representation


In [17]:
def train_model(
    model,
    train_loader,
    num_epochs=10,
    lr=1e-3,
    device='cuda' if torch.cuda.is_available() else 'cpu'
):
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    # Setup TensorBoard writer
    writer = SummaryWriter(log_dir='runs/BiLSTM_experiment')

    global_step = 0
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for i, (padded_text, padded_audio, text_mask, audio_mask, labels) in enumerate(train_loader):
            padded_text = padded_text.to(device)
            padded_audio = padded_audio.to(device)
            text_mask = text_mask.to(device)
            audio_mask = audio_mask.to(device)
            labels = labels.to(device)

            # Forward pass
            logits = model(padded_text, text_mask, padded_audio, audio_mask)
            loss = criterion(logits, labels)

            # Backprop
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            global_step += 1

            # Log loss to TensorBoard every N steps
            if (i + 1) % 10 == 0:
                avg_loss = running_loss / 10
                writer.add_scalar("training_loss", avg_loss, global_step=global_step)
                running_loss = 0.0

        print(f"Epoch [{epoch+1}/{num_epochs}] completed.")

    writer.close()
    print("Training finished!")


In [18]:
def evaluate_model(model, val_loader, device='cuda' if torch.cuda.is_available() else 'cpu'):
    model.eval()
    correct = 0
    total = 0
    criterion = nn.CrossEntropyLoss()
    val_loss = 0.0

    with torch.no_grad():
        for padded_text, padded_audio, text_mask, audio_mask, labels in val_loader:
            padded_text = padded_text.to(device)
            padded_audio = padded_audio.to(device)
            text_mask = text_mask.to(device)
            audio_mask = audio_mask.to(device)
            labels = labels.to(device)

            logits = model(padded_text, text_mask, padded_audio, audio_mask)
            loss = criterion(logits, labels)
            val_loss += loss.item()

            preds = torch.argmax(logits, dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    avg_loss = val_loss / len(val_loader)
    accuracy = 100.0 * correct / total
    print(f"Validation Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%")
    return avg_loss, accuracy


In [20]:
sample_text, sample_audio, _ = train_dataset[0]
text_dim  = sample_text.shape[1]
audio_dim = sample_audio.shape[1]

model = BiLSTMClassifier(
    text_dim=text_dim,
    audio_dim=audio_dim,
    hidden_dim=128,
    num_layers=2,
    dropout=0.3,
    use_attention=True
  )

train_model(model, train_loader, num_epochs=200, lr=1e-4)

evaluate_model(model, test_loader)

Epoch [1/200] completed.
Epoch [2/200] completed.
Epoch [3/200] completed.
Epoch [4/200] completed.
Epoch [5/200] completed.
Epoch [6/200] completed.
Epoch [7/200] completed.
Epoch [8/200] completed.
Epoch [9/200] completed.
Epoch [10/200] completed.
Epoch [11/200] completed.
Epoch [12/200] completed.
Epoch [13/200] completed.
Epoch [14/200] completed.
Epoch [15/200] completed.
Epoch [16/200] completed.
Epoch [17/200] completed.
Epoch [18/200] completed.
Epoch [19/200] completed.
Epoch [20/200] completed.
Epoch [21/200] completed.
Epoch [22/200] completed.
Epoch [23/200] completed.
Epoch [24/200] completed.
Epoch [25/200] completed.
Epoch [26/200] completed.
Epoch [27/200] completed.
Epoch [28/200] completed.
Epoch [29/200] completed.
Epoch [30/200] completed.
Epoch [31/200] completed.
Epoch [32/200] completed.
Epoch [33/200] completed.
Epoch [34/200] completed.
Epoch [35/200] completed.
Epoch [36/200] completed.
Epoch [37/200] completed.
Epoch [38/200] completed.
Epoch [39/200] comple

(2.033155918121338, 61.904761904761905)